In [27]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import copy
import missingno as msno
from sklearn.cluster import KMeans
%matplotlib inline 
# 기본 경로 설정
os.chdir('C:/Users/gksru/Desktop/study/Lpoint/제6회 L.POINT Big Data Competition')
# 단위 표시 설정
pd.options.display.float_format = '{:.5f}'.format
# 폰트 설정
mpl.rc('font', family=' Malgun Gothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

warnings.filterwarnings('ignore')

In [2]:
online_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv')
online_data.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,7809,1,8,5,A03,20190913,01:16,2571103,nan,NaN,34.00000,2663.00000,DIRECT,NaN
1,7809,1,4,2,A03,20190913,01:14,2485909,nan,NaN,34.00000,2663.00000,DIRECT,NaN
2,7809,1,11,5,A03,20190913,01:17,2646597,nan,NaN,34.00000,2663.00000,DIRECT,NaN
3,7809,1,1,2,A03,20190913,00:46,788304,nan,NaN,34.00000,2663.00000,DIRECT,NaN
4,7809,1,9,5,A03,20190913,01:17,2617609,nan,NaN,34.00000,2663.00000,DIRECT,NaN


In [3]:
trade_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv')
trade_data.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1


In [4]:
demo_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')
demo_data.head()

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
1,2,F,30
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown


In [5]:
class_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')
class_data.head()

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,4,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,5,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades


In [6]:
online_data['sess_dt'] = online_data['sess_dt'].astype('str')
online_data['sess_dt'] = online_data['sess_dt'] + online_data['hit_tm']
online_data['sess_dt'] = online_data['sess_dt'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d%H:%M'))

In [70]:
trade_drop.dtypes

clnt_id       int64
trans_id      int64
trans_seq     int64
biz_unit     object
pd_c          int32
de_dt         int64
de_tm        object
buy_am        int64
buy_ct        int64
dtype: object

In [ ]:
trade_drop['pd_c'] = trade_drop['pd_c'].astype('int')
trade_merge = trade_drop.merge(class_data, on = 'pd_c')

In [13]:
trade_drop = trade_data[trade_data['pd_c'] != 'unknown']

## 구매한 소분류 품목의 개수로 클러스터링(엉망) 

In [71]:
trade_pivot_1 = trade_merge.pivot_table(index = 'clnt_id', columns = 'clac_nm3', aggfunc = 'sum', values = 'buy_ct').fillna(0)

In [72]:
trade_pivot_1.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000
12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
20,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
23,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [74]:
kmeans_1 = KMeans(n_clusters = 10)
kmeans_1.fit(trade_pivot_1.values)

KMeans(n_clusters=10)

In [75]:
kmeans_1.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [76]:
kmeans_1.predict(trade_pivot_1.values)

array([0, 0, 0, ..., 0, 0, 0])

In [77]:
trade_kmean_1 = pd.DataFrame(
    {
    'clnt_id': trade_pivot_1.index,
    'cluster': kmeans.predict(trade_pivot_1.values)
    }
)

In [89]:
trade_kmean_1['cluster'].value_counts()

0    11211
3       21
7       17
6       10
9        4
5        3
4        1
2        1
1        1
8        1
Name: cluster, dtype: int64

## 구매한 중분류 품목의 개수로 클러스터링
* 그래도 컬럼이 너무 많은데?
* 군집을 어떻게 만들지

In [79]:
trade_pivot_2 = trade_merge.pivot_table(index = 'clnt_id', columns = 'clac_nm2', aggfunc = 'sum', values = 'buy_ct').fillna(0)

In [80]:
trade_pivot_2.head()

clac_nm2,Adults' Bedding,Air Freshners / Dehumidifiers / Odor Eliminators,Air Purifiers / Humidifiers / Dehumidifiers,Alcoholic Beverage Sets,Animal Clinics,Arts / Crafts Supplies,Audios,Australian Imported Beefs,Automotive Replacement Repair / Maintanance Kits,Baby Foods,...,Women's Socks and Hosiery,Women's Special Materials Clothing,Women's Special Use Clothing,Women's Sport Shoes,Women's Underwear,Women's Upper Bodywear / Tops,Women's Wallets,Writing Pads,Writing Supplies,Yogurt
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
20,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
23,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000


In [81]:
kmeans_2 = KMeans(n_clusters = 10)
kmeans_2.fit(trade_pivot_2.values)

KMeans(n_clusters=10)

In [82]:
kmeans_2.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [84]:
kmeans_2.predict(trade_pivot_2.values)

array([0, 0, 0, ..., 0, 0, 0])

In [85]:
trade_kmean_2 = pd.DataFrame(
    {
    'clnt_id': trade_pivot_2.index,
    'cluster': kmeans_2.predict(trade_pivot_2.values)
    }
)

In [88]:
trade_kmean_2['cluster'].value_counts()

0    11202
6       25
7       22
9       10
5        4
8        4
4        1
3        1
2        1
1        1
Name: cluster, dtype: int64

## 구매한 대분류 품목의 개수로 클러스터링

In [90]:
trade_pivot_3 = trade_merge.pivot_table(index = 'clnt_id', columns = 'clac_nm1', aggfunc = 'sum', values = 'buy_ct').fillna(0)

In [91]:
trade_pivot_3.head()

clac_nm1,Automotive Products,Baby Foods,Ball Game / Field Sports,Bedding / Handicraft,Beverages,Birth Supplies / Baby Products,Books / Records / Instruments,Canned / Jarred Foods,Cell Phones / Accessories,Chilled Foods,...,Tableware / Cooking Utensils,Tenants / Restaurants,Tobacco,Tools / Safety Supplies,Toy,Travel / Leisure Services,Underwear / Socks and Hosiery / Homewear,Vegetables,Video / Audio System Electronics,Women's Clothing
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.00000,0.00000,0.00000,0.00000,26.00000,0.00000,0.00000,1.00000,0.00000,2.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,2.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4.00000,0.00000,0.00000
12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
20,0.00000,0.00000,0.00000,0.00000,13.00000,0.00000,0.00000,1.00000,0.00000,3.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,13.00000,0.00000,0.00000
23,0.00000,0.00000,0.00000,0.00000,5.00000,0.00000,0.00000,0.00000,0.00000,10.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,11.00000,0.00000,0.00000


In [92]:
kmeans_3 = KMeans(n_clusters = 10)
kmeans_3.fit(trade_pivot_3.values)

KMeans(n_clusters=10)

In [93]:
kmeans_3.predict(trade_pivot_3.values)

array([0, 0, 0, ..., 0, 0, 0])

In [94]:
trade_kmean_3 = pd.DataFrame(
    {
    'clnt_id': trade_pivot_3.index,
    'cluster': kmeans_3.predict(trade_pivot_3.values)
    }
)

In [96]:
trade_kmean_3['cluster'].value_counts()

0    11126
4       79
2       32
3       21
8        4
9        3
7        2
6        2
5        1
1        1
Name: cluster, dtype: int64